## Import required libraries

In [1]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

# Part 1: Getting and cleaning the data

## Scraping Wikipedia page to get Toronto Neighbourhoods dataframe

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
r = requests.get(url)

In [4]:
dfs=pd.read_html(r.text, header=0)

In [5]:
toronto_neis = dfs[0]
toronto_neis.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Let's clean the dataframe

In [6]:
toronto_neis.shape

(180, 3)

In [7]:
toronto_neis['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [8]:
toronto_neis = toronto_neis[toronto_neis.Borough != 'Not assigned']
toronto_neis.shape

(103, 3)

In [9]:
toronto_neis.loc[toronto_neis['Neighbourhood'] == 'Not Assigned']

,Postal Code,Borough,Neighbourhood


In [10]:
toronto_neis.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
# note - some hoods with different postal codes have the same names:
print("Maximum counts per Neighbourhood is " + str(toronto_neis['Neighbourhood'].value_counts().max()))
print(" ")
print(toronto_neis['Neighbourhood'].value_counts(ascending=False).head())
print(" ")
print("Maximum counts per Postal Code is " + str(toronto_neis['Postal Code'].value_counts().max()))

Maximum counts per Neighbourhood is 4
 
Downsview                                                                  4
Don Mills                                                                  2
Toronto Dominion Centre, Design Exchange                                   1
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens    1
Harbourfront East, Union Station, Toronto Islands                          1
Name: Neighbourhood, dtype: int64
 
Maximum counts per Postal Code is 1


## Result for Part 1

In [12]:
toronto_neis.shape

(103, 3)

# Part 2: Geocoding

## Let's have coordinates in our neighbourhoods dataframe

In [13]:
geo_csv = pd.read_csv('Geospatial_Coordinates.csv')

In [14]:
geo_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
toronto_neis_geocoded = pd.merge(toronto_neis, geo_csv, on='Postal Code')

## Result for Part 2

In [16]:
toronto_neis_geocoded.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [17]:
toronto_neis_geocoded.shape

(103, 5)

# Part 3: Exloring and Clustering

## Let's narrow down our exploration area

In [18]:
toronto_neis_geocoded = toronto_neis_geocoded[toronto_neis_geocoded['Borough'].str.contains('Toronto')]

In [19]:
toronto_neis_geocoded.shape

(39, 5)

## Let's see the map

In [20]:
#Toronto coordinates
address = 'Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [41]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_neis_geocoded['Latitude'], toronto_neis_geocoded['Longitude'], toronto_neis_geocoded['Borough'], toronto_neis_geocoded['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Forsquare API credentials

In [42]:
CLIENT_ID = '?' # your Foursquare ID
CLIENT_SECRET = '?' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: ?
CLIENT_SECRET:?


## Explore Neighbourhoods in Toronto

### Function to get nearby venues per hood

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Search for venues

#### 500m search radius

In [24]:
toronto_venues = getNearbyVenues(names=toronto_neis_geocoded['Neighbourhood'],
                                   latitudes=toronto_neis_geocoded['Latitude'],
                                   longitudes=toronto_neis_geocoded['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### Let's check the size of the resulting dataframe

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(1621, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Let's see the minimum returned venues per hood

In [26]:
toronto_venues['Neighbourhood'].value_counts(ascending=True).head(10)

Roselawn                                                                                                       3
Moore Park, Summerhill East                                                                                    3
Lawrence Park                                                                                                  3
Forest Hill North & West, Forest Hill Road Park                                                                4
The Beaches                                                                                                    4
Rosedale                                                                                                       4
Davisville North                                                                                               8
Parkdale, Roncesvalles                                                                                        14
Dufferin, Dovercourt Village                                                                    

#### We need more for clustering. Let's increase search radius to 1000m

In [27]:
del(toronto_venues)
toronto_venues = getNearbyVenues(names=toronto_neis_geocoded['Neighbourhood'],
                                   latitudes=toronto_neis_geocoded['Latitude'],
                                   longitudes=toronto_neis_geocoded['Longitude'],
                                 radius=1000
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(3181, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [29]:
toronto_venues['Neighbourhood'].value_counts(ascending=True).head(10)

Lawrence Park                                                                                                  8
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport    14
Rosedale                                                                                                      21
Roselawn                                                                                                      23
St. James Town, Cabbagetown                                                                                   38
North Toronto West, Lawrence Park                                                                             42
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                          48
Forest Hill North & West, Forest Hill Road Park                                                               48
Moore Park, Summerhill East                                                                     

#### Good enough. Is any hood lost?

In [30]:
# there were 39 hoods in narrowed down area. With venues:
toronto_venues['Neighbourhood'].nunique()

39

## Analyze Each Neighbourhood

#### One hot encoding

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
toronto_onehot.shape

(3181, 270)

#### Frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,Berczy Park,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.020000,0.00,...,0.00,0.00,0.010000,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.01,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,0.00,...,0.00,0.00,0.020000,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.00,0.000000,0.020833,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.071429,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.00
4,Central Bay Street,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,0.01,...,0.00,0.01,0.020000,0.00000,0.00,0.000000,0.00,0.000000,0.020000,0.00
5,Christie,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,0.00,...,0.00,0.00,0.020000,0.00000,0.00,0.010000,0.00,0.010000,0.000000,0.00
6,Church and Wellesley,0.00,0.000000,0.010000,0.00,0.00,0.00,0.00,0.010000,0.00,...,0.00,0.00,0.010000,0.00000,0.01,0.000000,0.00,0.000000,0.020000,0.00
7,"Commerce Court, Victoria Hotel",0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.020000,0.00,...,0.00,0.00,0.020000,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.00
8,Davisville,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.010000,0.01000,0.00,0.010000,0.00,0.010000,0.010000,0.00
9,Davisville North,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,...,0.00,0.00,0.010000,0.01000,0.00,0.010000,0.00,0.010000,0.020000,0.00


In [34]:
toronto_grouped.shape

(39, 270)

#### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
         venue  freq
0  Coffee Shop  0.12
1        Hotel  0.06
2         Café  0.06
3   Restaurant  0.05
4         Park  0.04


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.08
1          Bar  0.05
2   Restaurant  0.05
3  Coffee Shop  0.05
4       Bakery  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0         Coffee Shop  0.08
1                Park  0.08
2         Pizza Place  0.06
3             Brewery  0.06
4  Italian Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0              Café  0.14
1       Coffee Shop  0.14
2   Harbor / Marina  0.14
3    Scenic Lookout  0.07
4  Sculpture Garden  0.07


----Central Bay Street----
              venue  freq
0       Coffee Shop  0.11
1  Sushi Restaurant  0.03
2              Park  0.03
3   

#### Let's get dataframe with 10 most common venues per hood

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Park,Japanese Restaurant,Beer Bar,Bakery,Gastropub,Gym
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Tibetan Restaurant,Gift Shop,Vegetarian / Vegan Restaurant,Gym
2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Brewery,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Burrito Place,Italian Restaurant,Electronics Store,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Coffee Shop,Harbor / Marina,Sculpture Garden,Dog Run,Park,Dance Studio,Track,Garden,Airport
4,Central Bay Street,Coffee Shop,Clothing Store,Gastropub,Park,Café,Sushi Restaurant,Yoga Studio,Breakfast Spot,Diner,Theater


## Cluster Neighborhoods

#### K-means clustering with 5 clusters

In [38]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 2, 3, 2, 4, 2, 0, 2, 2])

#### Merge cluster labels with top 10 venues per hood dataframe

In [39]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbourhoods_venues_sorted

# merge toronto_grouped with toronto_neis_geocoded to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_neis_geocoded.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
0,0,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Park,Japanese Restaurant,Beer Bar,Bakery,Gastropub,Gym,M5E,Downtown Toronto,43.644771,-79.373306
1,4,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Tibetan Restaurant,Gift Shop,Vegetarian / Vegan Restaurant,Gym,M6K,West Toronto,43.636847,-79.428191
2,2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Brewery,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Burrito Place,Italian Restaurant,Electronics Store,Farmers Market,M7Y,East Toronto,43.662744,-79.321558
3,3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Coffee Shop,Harbor / Marina,Sculpture Garden,Dog Run,Park,Dance Studio,Track,Garden,Airport,M5V,Downtown Toronto,43.628947,-79.394420
4,2,Central Bay Street,Coffee Shop,Clothing Store,Gastropub,Park,Café,Sushi Restaurant,Yoga Studio,Breakfast Spot,Diner,Theater,M5G,Downtown Toronto,43.657952,-79.387383


## Result for Part 3

#### Finally, let's visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters